In [3]:
import warnings
from typing import List, Optional, Dict
from langchain_core.pydantic_v1 import BaseModel, Field

from langchain_groq import ChatGroq
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer

from langgraph.graph import END, StateGraph
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

warnings.filterwarnings("ignore", category=FutureWarning)

c:\Users\tshar\anaconda3\envs\LegalDocRAG\Lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\tshar\anaconda3\envs\LegalDocRAG\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, tran

In [4]:
es_client = Elasticsearch(
    hosts="https://512baa73334c4127ade77bc2dfa2ef02.eastus2.azure.elastic-cloud.com:443",
    basic_auth=("elastic", "XRzYOABuG17Yv32UKAFySw41")
)

print(f"Elastic Search Connection successful: {es_client.ping()}")

Elastic Search Connection successful: True


In [5]:
llm = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192", #"llama-3.1-8b-instant",
    # api_key="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",
    api_key="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"
)

print("LLM Model loaded")

LLM Model loaded


In [6]:
llm("hello")

C:\Users\tshar\AppData\Local\Temp\ipykernel_488\173431691.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm("hello")


TypeError: Got unknown type h

In [217]:
model_name = "nomic-ai/nomic-embed-text-v1"
model = SentenceTransformer(model_name, trust_remote_code=True)

print("Embedding Model loaded")

<All keys matched successfully>


Embedding Model loaded


In [218]:
from langchain_core.tools import tool
from typing import List, Dict, Union

In [219]:
@tool("query_embedding_creation")
def embedding_creation(input_texts: Union[str, List[str]], ) -> List[float]:
    """
    Create embeddings for the input query (input_texts) using a given embedding model (model).

    Args:
        input_texts (Union[str, List[str]]): The input text(s) for which to create embeddings.

    Returns:
        List[float]: The generated embeddings as a list.
    """
    model_name = "nomic-ai/nomic-embed-text-v1"
    model = SentenceTransformer(model_name, trust_remote_code=True)
    
    # Ensure input_texts is a list
    if isinstance(input_texts, str):
        input_texts = [input_texts]

    # Validate input_texts
    if not isinstance(input_texts, list) or not all(isinstance(text, str) for text in input_texts):
        raise ValueError("input_texts must be a string or a list of strings.")

    # Create embeddings
    embedding = model.encode(input_texts)

    return embedding.tolist()[0]

In [220]:
query = "List of Board of Trustees of Mohamed bin Zayed University?"

query_embedding = embedding_creation({"input_texts": query, "model": model})
print(query_embedding)

<All keys matched successfully>


[0.005762419663369656, 0.010981039144098759, 0.016079256311058998, 0.002850219840183854, 0.04750065505504608, 0.04219432920217514, 0.00873507745563984, 0.0015295300399884582, 0.011511830613017082, -0.009813910350203514, -0.022294336929917336, 0.014243053272366524, 0.06803479790687561, 0.008896722458302975, -0.025524156168103218, -0.0338205024600029, 0.027076980099081993, -0.007133655250072479, -0.011596969328820705, -0.04390624165534973, -0.09026139974594116, -0.033539753407239914, -0.03986220806837082, 0.03444072604179382, 0.07740191370248795, 0.0030991253443062305, 0.024537885561585426, 0.02644938789308071, -0.06308671832084656, 0.005705669522285461, -0.010499754920601845, -0.002413531532511115, 0.04761940613389015, -0.021741187199950218, -0.019581953063607216, -0.018175622448325157, 0.09096667170524597, 0.021573221310973167, 0.04502426087856293, 0.01935414783656597, 0.040562085807323456, -0.013509991578757763, 0.007130123674869537, -0.022785509005188942, -0.0012931351084262133, 0.00

In [222]:
from langgraph.prebuilt import create_react_agent

tools = [embedding_creation]
agent_executor = create_react_agent(llm, tools)

In [223]:
from langchain_core.messages import HumanMessage, AIMessage

response = agent_executor.invoke(
    input = {
        "messages": [
            AIMessage(content="You are a helpful Agent for creating embedding for the user input. You can use 'embedding_creation' for embedding creation purpose "),
            HumanMessage(content="List of Board of Trustees of Mohamed bin Zayed University?")
            ]
    },
    
)

print(response["messages"])

<All keys matched successfully>


BadRequestError: Error code: 400 - {'error': {'message': 'Please reduce the length of the messages or completion.', 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [208]:
for key, value in response.items():
    print(f"{key}: {value}\n")

messages: [AIMessage(content="You are a helpful Agent for creating embedding for the user input. You can use 'embedding_creation' for embedding creation purpose ", additional_kwargs={}, response_metadata={}, id='5f8b4e26-f0f9-4990-94b7-8c48ec542f80'), HumanMessage(content='List of Board of Trustees of Mohamed bin Zayed University?', additional_kwargs={}, response_metadata={}, id='e8bd38c7-f5c6-49ba-8086-9ff1cbd910f9'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1mww', 'function': {'arguments': '{"input_texts":"List of Board of Trustees of Mohamed bin Zayed University"}', 'name': 'query_embedding_creation'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 1048, 'total_tokens': 1103, 'completion_time': 0.157427979, 'prompt_time': 0.118355589, 'queue_time': 0.004719389000000004, 'total_time': 0.275783568}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_87cbfbbc4d', 'finish_reason': 'tool_calls', 'log

In [209]:
response

{'messages': [AIMessage(content="You are a helpful Agent for creating embedding for the user input. You can use 'embedding_creation' for embedding creation purpose ", additional_kwargs={}, response_metadata={}, id='5f8b4e26-f0f9-4990-94b7-8c48ec542f80'),
  HumanMessage(content='List of Board of Trustees of Mohamed bin Zayed University?', additional_kwargs={}, response_metadata={}, id='e8bd38c7-f5c6-49ba-8086-9ff1cbd910f9'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1mww', 'function': {'arguments': '{"input_texts":"List of Board of Trustees of Mohamed bin Zayed University"}', 'name': 'query_embedding_creation'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 1048, 'total_tokens': 1103, 'completion_time': 0.157427979, 'prompt_time': 0.118355589, 'queue_time': 0.004719389000000004, 'total_time': 0.275783568}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_87cbfbbc4d', 'finish_reason': 'tool_calls

In [210]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate

In [211]:
@tool("query_embedding_creation")
def embedding_creation(input_texts: Union[str, List[str]], ) -> List[float]:
    """
    Create embeddings for the input query (input_texts) using a given embedding model (model).

    Args:
        input_texts (Union[str, List[str]]): The input text(s) for which to create embeddings.

    Returns:
        List[float]: The generated embeddings as a list.
    """
    model_name = "nomic-ai/nomic-embed-text-v1"
    model = SentenceTransformer(model_name, trust_remote_code=True)
    
    # Ensure input_texts is a list
    if isinstance(input_texts, str):
        input_texts = [input_texts]

    # Validate input_texts
    if not isinstance(input_texts, list) or not all(isinstance(text, str) for text in input_texts):
        raise ValueError("input_texts must be a string or a list of strings.")

    # Create embeddings
    embedding = model.encode(input_texts)

    return embedding.tolist()[0][:200]

In [212]:
tools = [embedding_creation]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the given tool to create embedding",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [213]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...

Invoking: `query_embedding_creation` with `{'input_texts': 'LangChain'}`




<All keys matched successfully>


[-0.03282121568918228, -0.004796232562512159, -0.01032858993858099, -0.06253907829523087, -0.0013192269252613187, -0.05588463321328163, -0.0663987547159195, 0.02573421038687229, -0.048716943711042404, -0.02377275377511978, -0.07333897799253464, 0.05918097868561745, 0.03603370860219002, 0.019900672137737274, 0.019771214574575424, 0.011526651680469513, 0.03354062885046005, -0.02980607934296131, -0.01167558878660202, 0.004660237114876509, -0.03980294615030289, 0.05243140831589699, -0.01735762506723404, -0.03477517515420914, 0.12092906981706619, -0.04315202683210373, 0.014984636567533016, 0.011364752426743507, -0.0643455758690834, -0.016489744186401367, 0.03134414553642273, -0.005598294083029032, -0.029684729874134064, -0.02569323033094406, -0.08699178695678711, -0.05372743308544159, 0.04860115051269531, -0.00902702659368515, -0.00425192853435874, 0.05092988163232803, 0.07784964889287949, 0.05019170418381691, 0.007920279167592525, 0.009581110440194607, 0.060844749212265015, -0.001108806929

{'input': 'what is LangChain?',
 'output': 'The provided embedding is a numerical representation of the input text "LangChain". This embedding can be used as input to other models or algorithms for further processing.'}

In [109]:
@tool("elasticsearch")
def hybrid_elasticsearch_information_retrival(
        es_client,
        index_name: str,
        query_keys: List[str],
        query_vector: List[float],
        text_fields: List[str],
        vector_field: str,
        num_candidates: int = 100,
        num_results: int = 10,
    ) -> Dict:
    """
    Performs a hybrid vector search in Elasticsearch knowledgebase

    Args:
        es_client: Connection to the search backend.
        index_name (str): The name of the index to search.
        query_keys (List[str]): The keys to search in the text fields.
        query_vector (List[float]): The vector representation of the query.
        text_fields (List[str]): The text fields to match against.
        vector_field (str): The name of the field containing vector data.
        num_candidates (int): Number of candidates to consider (default: 100).
        num_results (int): Number of results to return (default: 10).

    Returns:
        Dict: The search response and the constructed query body.
    """

    if not isinstance(query_keys, list) or not all(isinstance(k, str) for k in query_keys):
        raise ValueError("query_keys must be a list of strings.")
    if not isinstance(query_vector, list) or not all(isinstance(v, float) for v in query_vector):
        raise ValueError("query_vector must be a list of floats.")
    if not isinstance(text_fields, list) or not all(isinstance(f, str) for f in text_fields):
        raise ValueError("text_fields must be a list of strings.")
    
    response = es_client.search(
    index=index_name,
    size=5,
    knn={
        "field": vector_field,
        "query_vector": query_vector,
        "k": num_results,
        "num_candidates": num_candidates,
    },
    query={
        "bool": {
            "must": [
                {
                    "multi_match": {
                        "query": " ".join(query_keys),
                        "fields": text_fields,
                        "type": "best_fields",
                        "operator": "or"
                    }
                }
            ],
            "should": [
                {
                    "script_score": {
                        "query": {"match_all": {}},
                        "script": {
                            "source": """
                                double vector_score = cosineSimilarity(params.query_vector, params.vector_field) + 1.0;
                                double text_score = _score;
                                return 0.7 * vector_score + 0.3 * text_score;
                            """,
                            "params": {
                                "query_vector": query_vector,
                                "vector_field": vector_field
                            }
                        }
                    }
                }
            ]
        }
    },
    # rank={"rrf": {}},
    )

    return response


In [110]:
unique_enriched_keys = ['higher education', 'council members', 'abu dhabi', 'board of directors', 'uae', 'university members', 'university', 'mohamed bin zayed university', 'Trustees', 'Mohamed bin Zayed University', 'educational institution', 'Mohamed bin Zayed University Abu Dhabi', 'bin zayed', 'Board of Trustees', 'mohamed bin zayed', 'university trustee', 'University Board of Trustees', 'member of the board', 'Abu Dhabi University', 'academic board', 'administration', 'university administration', 'trustee members', 'mohamed bin', 'trustee', 'MBZU', 'officials', 'Mohamed bin Zayed', 'university board', 'governing body', 'board members', 'university governance', 'University of Abu Dhabi', 'university board of directors', 'University Trustees']
query_vector = [0.03211868181824684, 0.016570353880524635, 0.008851059712469578, 0.009300854988396168, 0.0542679987847805, 0.04002238065004349, -0.005691547878086567, 0.021677056327462196, -0.011182517744600773, 0.010330944322049618, -0.026781562715768814, 0.009384237229824066, 0.03477415442466736, -0.018618278205394745, 0.012149563059210777, -0.02137056365609169, 0.009613710455596447, 0.010713585652410984, -0.01866135001182556, -0.03623474761843681, -0.09695301204919815, -0.028328677639365196, -0.027456248179078102, 0.005014834925532341, 0.10671402513980865, -0.034579236060380936, 0.021841159090399742, 0.05384795740246773, -0.04044593870639801, 0.012115568853914738, 0.01494857668876648, 0.01123364269733429, 0.04732108861207962, -0.05608712136745453, -0.015513290651142597, -0.017483074218034744, 0.09898950904607773, 0.018279092386364937, 0.044104915112257004, 0.029281992465257645, 0.019735923036932945, -0.01147719006985426, 0.008299834094941616, -0.033253662288188934, 0.0013564650435000658, 0.007097032852470875, 0.03667600080370903, 0.009773015044629574, 0.06464532762765884, 0.008989903144538403, -0.025265086442232132, -0.008459852077066898, 0.04141782224178314, -0.057720817625522614, -0.023942558094859123, -0.0012918369611725211, -0.046291325241327286, -0.001739715808071196, 0.03197962045669556, 0.020184919238090515, -0.006008827593177557, 0.02551880106329918, -0.04438505321741104, 0.009595535695552826, 0.032179493457078934, -0.0638372078537941, 0.009739357978105545, 0.01634180173277855, 0.00854454841464758, -0.015537189319729805, 0.028763217851519585, -0.027728717774152756, 0.042139992117881775, 0.061969559639692307, -0.003262832760810852, -0.015061802230775356, 0.029661623761057854, -0.01893465965986252, 0.03739774599671364, 0.009337134659290314, 0.055782612413167953, -0.02710302732884884, 0.023627014830708504, 0.044614583253860474, -0.024294914677739143, 0.01717737503349781, -0.015227516181766987, 0.020858054980635643, -0.041436415165662766, 0.0327644906938076, 0.02038579247891903, -0.01741967909038067, 0.05095139518380165, 0.10007336735725403, -0.01600261777639389, 0.011111607775092125, 0.004632933996617794, -0.03181321546435356, 0.008916730992496014, -0.0016561975935474038, 0.03080027736723423, 0.0677623599767685, 0.004942312370985746, 0.015054825693368912, 0.10173123329877853, -0.0028040281031280756, 0.04748602584004402, -0.025285346433520317, 0.013455264270305634, 0.005353996530175209, -0.07264965772628784, 0.025472775101661682, 0.0007442070636898279, 0.04433793947100639, -0.009777734987437725, 0.043110594153404236, 0.006211026571691036, -0.03339019790291786, -0.05213526263833046, 0.04099864885210991, -0.028839338570833206, -0.0574931763112545, -0.01900605857372284, 0.0033397504594177008, -0.011297477409243584, -0.006737115327268839, -0.06382542848587036, -0.023557312786579132, 0.014646092429757118, 0.05509767681360245, -0.0028322392608970404, 0.037040531635284424, -0.017756683751940727, -0.04022859036922455, -0.025355197489261627, 0.04381059482693672, -0.04512106254696846, -0.05502403527498245, 0.006692149210721254, -0.00043645116966217756, 0.010989924892783165, 0.014078806154429913, -0.010947790928184986, 5.2957984735257924e-05, 0.005738339386880398, -0.06871048361063004, 0.057603321969509125, -0.016632432118058205, -0.03949727490544319, 0.01132824644446373, -0.014548094943165779, 0.023563731461763382, -0.0051338691264390945, 0.06369093060493469, -0.014615853317081928, -0.04436102882027626, -0.005309425760060549, 0.027443142607808113, 0.042502544820308685, 0.03213255852460861, 0.03778837248682976, 0.04863018915057182, -0.024894678965210915, 0.004950589034706354, -0.04668649286031723, -0.029437776654958725, 0.025599030777812004, 0.02001621201634407, 0.05678758770227432, 0.09712989628314972, -0.005747823044657707, -0.04495251551270485, 0.015817193314433098, -0.0028926199302077293, 0.01618950255215168, 0.016410529613494873, -0.03330681845545769, -0.016583003103733063, 0.002014991594478488, 0.0241237822920084, 0.011860541068017483, 0.0035419161431491375, 0.05796501040458679, 0.017132161185145378, -0.023965325206518173, -0.027033334597945213, -0.006139164324849844, -0.05546824261546135, 0.0028766407631337643, -0.003408751916140318, -0.03745030611753464, 0.035643476992845535, 0.011327415704727173, -0.05029503256082535, -0.027631431818008423, -0.02369857020676136, 0.05409557372331619, -0.006161851342767477, 0.0333687923848629, -0.0181821770966053, -0.0049537369050085545, -0.03680047392845154, -0.04343210160732269, -0.05666032060980797, 0.007264694198966026, 0.045835234224796295, 0.05416274443268776, 0.02127101644873619, -0.0382843017578125, 0.01354588009417057, 0.01563362590968609, 0.04134070873260498, -0.02027013897895813, -0.06146935373544693, -0.03664775192737579, -0.04932596907019615, 0.013044959865510464, -0.038778673857450485, -0.0008248884696513414, 0.015468195080757141, 0.013236437924206257, -0.04249086230993271, -0.0016049259575083852, -0.022561009973287582, 0.05943364277482033, -0.013018714264035225, -0.03548837825655937, -0.11064356565475464, -0.06901924312114716, -0.004803324118256569, -0.038651350885629654, -0.07990573346614838, 0.0704614520072937, 0.0745759978890419, -0.013156311586499214, -0.021011561155319214, -0.006034495308995247, 0.02190917357802391, 0.05488809198141098, 0.0319073311984539, 0.04396094009280205, 0.06502272933721542, 0.021841827780008316, 0.018396267667412758, -0.04823649302124977, -0.07851585745811462, 0.0033358177170157433, -0.03157683089375496, -0.0004370624665170908, 0.023852000012993813, 0.00922276359051466, -0.01516437903046608, 0.06660591810941696, 0.06405678391456604, 0.0006420327699743211, -0.009918090887367725, -0.050896480679512024, 0.007305134553462267, 0.02852681837975979, -0.028799738734960556, 0.02920990251004696, -0.08900098502635956, 0.0447583831846714, 0.035707972943782806, 0.008932406082749367, -0.025823069736361504, -0.03469553962349892, 0.020253274589776993, -0.03455435484647751, 0.025309672579169273, 0.017499562352895737, 0.06311392039060593, -0.02075958624482155, 0.014290088787674904, 0.014562946744263172, -0.019594067707657814, 0.015326260589063168, 0.003946969285607338, -0.094121053814888, 0.02806815132498741, 0.03584219142794609, 0.005370330065488815, -0.0405326671898365, 0.016551801934838295, -0.005441212560981512, 0.0036629289388656616, 0.07733670622110367, -0.059148386120796204, -0.06075137108564377, 0.03155236691236496, 0.005350938532501459, -0.04470714181661606, 0.06823548674583435, 0.07937739044427872, 0.012905685231089592, 0.02459845505654812, 0.05288755148649216, 0.030679207295179367, -0.05413549020886421, 0.02989170514047146, -0.03536544367671013, 0.015501076355576515, -0.04503275081515312, -0.002917437581345439, 0.018525253981351852, 0.04307589307427406, 0.023358119651675224, 0.04178561642765999, 0.0398496575653553, -0.034675482660532, -0.028399953618645668, -0.017772650346159935, -0.0029959185048937798, -0.006312210112810135, 0.005471975542604923, 0.008875997737050056, -0.00981446634978056, 0.002341577550396323, -0.007897714152932167, 0.04078017547726631, -0.0018288323190063238, 0.03346240520477295, 0.020754799246788025, -0.01913805492222309, 0.01533424574881792, 0.019235635176301003, 0.03587081655859947, -0.017144499346613884, -0.021653005853295326, -0.02265186421573162, -0.010077707469463348, 0.026143571361899376, -0.044174011796712875, 0.006975241005420685, 0.0261519942432642, -0.012682096101343632, -0.025535376742482185, 0.007036138791590929, -0.012386192567646503, -0.05325809493660927, 0.04275156185030937, 0.014318970963358879, -0.030632467940449715, -0.050540801137685776, 0.000666244188323617, -0.03937027230858803, -0.028902918100357056, -0.02205614373087883, -0.014991512522101402, -0.03304692730307579, -0.05965080484747887, -0.003567454870790243, -0.012146567925810814, 0.011026830412447453, 0.008800416253507137, 0.07300782203674316, 0.009999397210776806, -0.008697005920112133, 0.014887404628098011, -0.004075916018337011, 0.017725933343172073, -0.045178379863500595, 0.029828976839780807, 0.0018143110210075974, 0.0022050468251109123, 0.03138605132699013, 0.020793499425053596, -0.00463816849514842, 0.009510142728686333, -0.02639034017920494, -0.053247980773448944, -0.05464410409331322, 0.008673257194459438, -0.0005848041037097573, 0.07433241605758667, -0.030478226020932198, -0.03927260637283325, -0.054187435656785965, 0.007559115532785654, -0.01664789207279682, 0.0015531700337305665, 0.03641501069068909, 0.01430769544094801, 0.007709180936217308, 0.03304336965084076, 0.009638063609600067, -0.0808592438697815, -0.029730606824159622, 0.02013907954096794, 0.0013222062261775136, 0.002158969407901168, 0.001151291886344552, 0.004649544134736061, -0.014974999241530895, -0.044318899512290955, -0.03708018735051155, 0.020608197897672653, -0.01579780876636505, 0.025255883112549782, 0.01600300520658493, -0.029205432161688805, -0.05427553132176399, -0.020124046131968498, -0.018531078472733498, -0.026544850319623947, 0.007380746304988861, -0.034528035670518875, -0.09263831377029419, 0.04704772308468819, 0.06591767072677612, -0.017874160781502724, 0.005538317374885082, 0.01590435765683651, -0.0827891007065773, 0.012613316997885704, 0.0637272447347641, -0.006540944799780846, -0.014173957519233227, -0.0030631329864263535, 0.015269692055881023, 0.03348754718899727, 0.018940012902021408, -0.0440826453268528, -0.02477526105940342, -0.03704959899187088, -0.016499003395438194, -0.003662110073491931, 0.010217487812042236, 0.059057872742414474, -0.0025332048535346985, -0.012234794907271862, -0.005612586624920368, -0.005586793180555105, 0.04493355005979538, -0.034296128898859024, -0.021665628999471664, 0.12297461181879044, 0.030978834256529808, 0.007219205144792795, 0.004747201222926378, 0.0505693182349205, -0.01598837412893772, -0.06829410791397095, 0.0036805691197514534, 0.045780349522829056, 0.01850460097193718, 0.1301237791776657, -0.028502101078629494, 0.03185892850160599, 0.030296476557850838, 0.05227766931056976, -0.004238366615027189, 0.005080310627818108, 0.012785512953996658, 0.05266106128692627, -0.032647956162691116, -0.018764885142445564, -0.0698753073811531, 0.05363822355866432, 0.007631033658981323, 0.021755781024694443, 0.02681567892432213, -0.0208506491035223, 0.0540265254676342, -0.03315896540880203, -0.00886804610490799, 0.012128940783441067, 0.013639165088534355, 0.0031058357562869787, 0.0005544602172449231, 0.012090230360627174, 0.03308124840259552, 0.02761163003742695, -0.04412572830915451, -0.04466226324439049, -0.04224827513098717, -0.027072008699178696, -0.025847487151622772, 0.016872623935341835, 0.018215982243418694, 0.04046345129609108, 0.019760388880968094, -0.04028051346540451, -0.05425579845905304, -0.00406345259398222, 0.000865105539560318, 0.06925246864557266, 0.003657485591247678, 0.019372398033738136, -0.020748667418956757, -0.00608523515984416, 0.0037757628597319126, 0.022538291290402412, -0.026911543682217598, -0.032014183700084686, 0.008887278847396374, -0.016326677054166794, 0.006736636161804199, 0.0627131536602974, -0.014866087585687637, -0.023124298080801964, 0.02414812333881855, 0.009899109601974487, 0.012245191261172295, -0.008792475797235966, 0.04222780466079712, 0.0035357652232050896, -0.05370304733514786, -0.013292919844388962, 0.01166219636797905, -0.07311609387397766, -0.00455735856667161, 0.07537077367305756, 0.0486450269818306, 0.005981226917356253, -0.04718918725848198, 0.06828519701957703, 0.00459097558632493, -0.028460120782256126, 0.08825401216745377, 0.039086565375328064, 0.013077427633106709, -0.010893252678215504, 0.0013424622593447566, -0.053160298615694046, 0.028873875737190247, -0.0016045395750552416, -0.07116378098726273, 0.04184162989258766, 0.011745660565793514, 0.0101477624848485, -0.005107582081109285, 0.009470457211136818, -0.013893360272049904, 0.04452815651893616, -0.09098353236913681, 0.02893722988665104, -0.030083067715168, 0.016566460952162743, -0.00020549705368466675, 0.010698811151087284, 0.015839504078030586, 0.03322339430451393, 0.04576431214809418, 0.0009535568533465266, -0.036319926381111145, -0.004652883391827345, -0.005804162938147783, -0.03370290249586105, 0.01886606588959694, 0.04503466561436653, -0.009241983294487, -0.01650843769311905, -0.07721063494682312, -0.03575355187058449, 0.03313324972987175, -0.011843294836580753, -0.0035668411292135715, -0.03810373321175575, -0.0203818641602993, 0.01498655416071415, 0.004139945842325687, -0.01944395899772644, -0.035420022904872894, -0.04849577695131302, -0.07901675254106522, -0.0335879772901535, 0.0033013822976499796, -0.00991229247301817, -0.06856080889701843, -0.005205783061683178, -0.025653844699263573, -0.003975136671215296, -0.07095400243997574, -0.009116624481976032, -0.03474273532629013, -0.029950520023703575, -0.0029874551109969616, -0.009225517511367798, -0.04721393436193466, 0.00762996356934309, -0.0309445820748806, 0.004632368218153715, -0.006319069303572178, 0.055002328008413315, 0.0006125508807599545, -0.019963428378105164, -0.054065920412540436, 0.007823178544640541, 0.00024283047241624445, 0.009604002349078655, 0.023146428167819977, -0.055352188646793365, 0.02154727466404438, -0.05162955820560455, -0.010391978546977043, 0.043124306946992874, 0.053898319602012634, -0.004608894232660532, -0.026499062776565552, -0.023407695814967155, -0.006447725463658571, 0.009557828307151794, -0.0479591004550457, 0.06678721308708191, 0.08846233040094376, -0.04728381335735321, 0.013049717992544174, -0.00031383850728161633, -0.03353495895862579, 0.05349867418408394, -0.006780701689422131, -0.038525860756635666, -0.03372425213456154, -0.022834667935967445, 0.002837707055732608, 0.04271074756979942, -0.020817870274186134, -0.006910476367920637, -0.008848794735968113, 0.024400619789958, -0.006388518493622541, 0.007045558653771877, -0.027828652411699295, 0.009738742373883724, 0.007043351419270039, 0.008355947211384773, -0.03920845687389374, -0.02844770811498165, 0.0565083771944046, 0.026245854794979095, 0.04692935571074486, 0.014205851592123508, -0.07751207053661346, 0.01819736137986183, 0.07128238677978516, -0.06633704900741577, 0.018915046006441116, -0.0174509659409523, 0.04847477748990059, 0.0036890977062284946, -0.03510332480072975, 0.014197802171111107, 0.01853768900036812, 0.013237315230071545, -0.023262543603777885, -0.025998493656516075, 0.023204278200864792, -0.0019163062097504735, 0.0067272186279296875, -0.007114266976714134, 0.05425744131207466, -0.028144650161266327, 0.03026202879846096, -0.018959874287247658, -0.0019537564367055893, 0.018639162182807922, -0.01816614344716072, -0.023635875433683395, -0.04028518125414848, -0.02767314948141575, -0.0052647157572209835, -0.021961437538266182, -0.0013702368596568704, 0.026073357090353966, 0.01332450658082962, 0.029647568240761757, 0.007379408925771713, -0.03756769746541977, -0.011016227304935455, -0.03469103202223778, -0.06094302982091904, 0.005025292746722698, -0.02044273540377617, -0.07438337057828903, -0.02569110319018364, 0.0015852181240916252, 0.03960325941443443, 0.005066645331680775, -0.07148631662130356, 0.012123921886086464, 0.046503521502017975, 0.006099986843764782, -0.06904042512178421, 0.02393408492207527, 0.009379591792821884, 0.04093654453754425, -0.025945479050278664, 0.08683276176452637, -0.039808373898267746, 0.05317346751689911, -0.05277956277132034, -0.03189042955636978, -0.06773523986339569, 0.041202742606401443, -0.0451454296708107, -0.05885378271341324, 0.017554715275764465, -0.028381353244185448, -0.020923787727952003, 0.004858860280364752, 0.043096333742141724, 0.02072017267346382, 0.05009947344660759, 0.030045464634895325, -0.017257563769817352, 0.003915340639650822, 0.04708664491772652, -0.02637818455696106, -0.008951222524046898, -0.005632566288113594, -0.0457439161837101, 0.023242443799972534, 0.05758754909038544, 0.07076283544301987, -0.03562988340854645, -0.07815475016832352, -0.013117383234202862, 0.018045902252197266, 0.03512245789170265, 0.09770099073648453, 0.000148612423799932, -0.05969593673944473, 0.0016227111918851733, -0.0435103103518486, -0.034718118607997894, 0.013867583125829697, 0.01374496053904295, 0.032676178961992264, 0.012378785759210587, -0.05387958884239197, -4.557402280624956e-05, -0.04574921727180481, -0.0039167264476418495, -0.02723744884133339, 0.01070464588701725, 0.0011712670093402267, 0.026593707501888275, -0.02139207161962986, -0.02456577681005001, 0.06003309786319733, -0.0280089620500803, 0.017765088006854057, 0.03606628254055977, -0.019352979958057404, -0.0013409126549959183, -0.08850280195474625, -0.04115374758839607, 0.08274701237678528, 0.027290605008602142, -0.0012367747258394957, 0.03154689446091652, -0.054390307515859604, 0.05203576758503914, 0.0313761904835701, -0.01063582394272089, -0.014438476413488388, -0.000718628172762692, 0.007418609689921141, 0.0049532148987054825, 0.02335285022854805, -0.0567808635532856, 0.019400080665946007, 0.006176779046654701, 0.022027313709259033, 0.046731848269701004, -0.0278648491948843, -0.055586233735084534]

In [111]:
response = hybrid_elasticsearch_information_retrival(
    {
    "es_client": es_client,
    "index_name": "abudhabi-policies-legislations-v4",
    "query_keys": unique_enriched_keys,
    "query_vector": query_vector,
    # "query_vector": "Hello",
    "text_fields": ["content", "heading", "questions", "reference"],
    "vector_field": "embedded_content",
    "num_candidates": 25,
    "num_results": 3
    }
)


In [112]:
result = []
all_content = ""
for j in response['hits']['hits']:
    content = j["_source"]
    _result = {}
    _result["score"] = j["_score"]
    _result["heading"] = content['heading']
    _result["content"] = content['content']
    all_content = all_content + '\n' + content['content']
    _result["reference"] = content['reference']

    result.append(_result)

result

[{'score': 196.61223,
  'heading': 'executive council resolution no. (11) of 2023 concerning the reformation of the board of trustees of mohamed bin zayed university for humanities',
  'content': 'executive council resolution no. (11) of 2023 concerning the reformation of the board of trustees of mohamed bin zayed university for humanities having reviewed chairman of the executive council resolution no. (27) of 2020 concerning the formation of the board of trustees of mohamed bin zayed university for humanities. the executive council has decided the following: 1. the board of trustees of mohamed bin zayed university for humanities shall be reformed under the chairmanship of h.e. dr. mohammed rashid ahmed al hamli, and the membership of: • mubarak hamad mubarak al muhairi - vice chairman • mohammed hamza hassan al qassim. • maryam eid khamis al muhairi • abdullah aqueeda ali al muhairi • dr. omar habtoor theeb al-derei. • ghanem sultan ahmed al suwaidi. 2. the membership term of the boa

In [164]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

system_prompt = """
You are the Agent responsible for Retrieval Augmented Generation and you are the great AI decision maker.
Given the user's query you must decide what to do with it based on the
list of tools provided to you.

List of tools you are allowed to use:
 - Tool name: "embedding_creation" used for creating query embedding

You should aim to create the query embedding with "embedding_creation" tool and provide embedding vector as a result
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("assistant", "scratchpad: {scratchpad}"),
])

In [166]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator


class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

In [167]:
from langchain_core.messages import ToolCall, ToolMessage

tools=[
    embedding_creation,
    hybrid_elasticsearch_information_retrival,
]

# define a function to transform intermediate_steps from list
# of AgentAction to scratchpad string
def create_scratchpad(intermediate_steps: list[AgentAction]):
    research_steps = []
    for i, action in enumerate(intermediate_steps):
        if action.log != "TBD":
            # this was the ToolExecution
            research_steps.append(
                f"Tool: {action.tool}, input: {action.tool_input}\n"
                f"Output: {action.log}"
            )
    return "\n---\n".join(research_steps)

llm.bind_tools(tools)

oracle = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "scratchpad": lambda x: create_scratchpad(
            intermediate_steps=x["intermediate_steps"]
        ),
    }
    | prompt
    | llm
)

In [168]:
inputs = {
    "input": "How are you",
    "chat_history": [],
    "intermediate_steps": [],
}
out = oracle.invoke(inputs)
out

AIMessage(content=' []\n embeddings:  {\'query\': []}\n\nI\'ve received a query "How are you". Now, I\'ll create a query embedding using the "embedding_creation" tool.\n\n embeddings:  {\'query\': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 111, 'total_tokens': 203, 'completion_time': 0.262857143, 'prompt_time': 0.004278852, 'queue_time': 0.037783825, 'total_time': 0.267135995}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_7ab5f7e105', 'finish_reason': 'stop', 'logprobs': None}, id='run-1cefcca7-b676-47d1-a8dd-831f9378d72c-0', usage_metadata={'input_tokens': 111, 'output_tokens': 92, 'total_tokens': 203})

In [169]:
out.tool_calls[0]["name"]

IndexError: list index out of range

In [ ]:
tool_str_to_func = {
    "rag_search_filter": rag_search_filter,
    "rag_search": rag_search,
    "fetch_arxiv": fetch_arxiv,
    "web_search": web_search,
    "final_answer": final_answer
}

def run_tool(state: list):
    # use this as helper function so we repeat less code
    tool_name = state["intermediate_steps"][-1].tool
    tool_args = state["intermediate_steps"][-1].tool_input
    print(f"{tool_name}.invoke(input={tool_args})")
    # run tool
    out = tool_str_to_func[tool_name].invoke(input=tool_args)
    action_out = AgentAction(
        tool=tool_name,
        tool_input=tool_args,
        log=str(out)
    )
    return {"intermediate_steps": [action_out]}

In [170]:
@tool("math_calculation")
def calculate_sum(a: int, b:int) -> int:
    """This function calculates the sum of two numbers."""
    return a + b

In [172]:
from langgraph.prebuilt import create_react_agent

tools = [calculate_sum]
agent_executor = create_react_agent(llm, tools)

In [173]:
from langgraph.prebuilt import create_react_agent

tools = [calculate_sum]
agent_executor = create_react_agent(llm, tools)

from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="What is the sum of 5 and 10?")]})

response["messages"]

[HumanMessage(content='What is the sum of 5 and 10?', additional_kwargs={}, response_metadata={}, id='870bc69e-c51d-4638-b790-4a625d856857'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2f16', 'function': {'arguments': '{"a":5,"b":10}', 'name': 'math_calculation'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 932, 'total_tokens': 982, 'completion_time': 0.142857143, 'prompt_time': 0.041996544, 'queue_time': 0.028151825000000005, 'total_time': 0.184853687}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_2f30b0b571', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4e6ebf99-6414-4e99-8217-f04477035ca0-0', tool_calls=[{'name': 'math_calculation', 'args': {'a': 5, 'b': 10}, 'id': 'call_2f16', 'type': 'tool_call'}], usage_metadata={'input_tokens': 932, 'output_tokens': 50, 'total_tokens': 982}),
 ToolMessage(content='15', name='math_calculation', id='d8f13a78-dfe0-40cf-b6f9-ac0630d6a02e', t